<a href="https://colab.research.google.com/github/preetamjumech/LLM/blob/main/MachineTranslation_Transformer_17_11_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras

keras.__version__

'3.5.0'

In [2]:
import re
import string
import random
import pathlib

import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

In [3]:
with open('/content/fra.txt') as f:
    lines = f.read().split("\n")[:-1]

text_pairs = []

for line in lines:
    eng, fra = line.split("\t")
    fra = "[start] " + fra + " [end]"

    text_pairs.append((eng, fra))

In [4]:
for _ in range(10):
    print(random.choice(text_pairs))

('I want to see your boss.', '[start] Je veux voir votre patronne. [end]')
('I need you to fix this.', "[start] J'ai besoin que vous corrigiez ceci. [end]")
('Cut the cloth diagonally.', "[start] Coupez l'étoffe en diagonale. [end]")
('They all cheated.', '[start] Elles ont toutes triché. [end]')
('See to it that all the doors are locked before you go out.', '[start] Vérifiez bien que toutes les portes sont fermées à clef avant que vous ne partiez. [end]')
('I corrected it.', "[start] Je l'ai corrigée. [end]")
("I don't know if I can stay.", "[start] J'ignore si je peux rester. [end]")
('Our neighbor has broken a rib.', "[start] Notre voisine s'est cassée une côte. [end]")
('May I eat a little of it?', '[start] Est-ce que je peux en manger un peu\u202f? [end]')
('Can I borrow your car?', '[start] Puis-je emprunter votre voiture\u202f? [end]')


In [5]:
text_pairs = text_pairs[:1000]
random.shuffle(text_pairs)

num_val_samples = int(0.15 * len(text_pairs))

num_train_samples = len(text_pairs) - 2 * num_val_samples

train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

1000 total pairs
700 training pairs
150 validation pairs
150 test pairs


In [6]:
strip_chars = string.punctuation
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 64

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)

    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


eng_vectorization = TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length,
)

fra_vectorization = TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length + 1,
    standardize = custom_standardization,
)


In [7]:
train_eng_texts = [pair[0] for pair in train_pairs]
train_fra_texts = [pair[1] for pair in train_pairs]

eng_vectorization.adapt(train_eng_texts)
fra_vectorization.adapt(train_fra_texts)

In [8]:
vocab_inp_size = len(eng_vectorization.get_vocabulary())

vocab_inp_size

338

In [9]:
eng_vectorization.get_vocabulary()[:15]

['',
 '[UNK]',
 'i',
 'im',
 'it',
 'go',
 'me',
 'tom',
 'we',
 'its',
 'up',
 'be',
 'get',
 'you',
 'this']

In [10]:
vocab_out_size = len(fra_vectorization.get_vocabulary())

vocab_out_size

670

In [11]:
fra_vectorization.get_vocabulary()[:15]

['',
 '[UNK]',
 '[start]',
 '[end]',
 'je',
 'suis',
 'nous',
 'tom',
 'cest',
 'me',
 'jai',
 'il',
 'ça',
 'est',
 'de']

In [12]:
def format_dataset(eng, fra):
    eng = eng_vectorization(eng)
    fra = fra_vectorization(fra)

    # ({english tokens, french tokens}, french tokens shifted by 1)
    return (
        {
            "encoder_inputs": eng,
            "decoder_inputs": fra[:, :-1],
        },
        fra[:, 1:],
    )


def make_dataset(pairs):
    eng_texts, fra_texts = zip(*pairs)

    eng_texts = list(eng_texts)
    fra_texts = list(fra_texts)

    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, fra_texts))

    dataset = dataset.batch(batch_size)

    dataset = dataset.map(format_dataset)

    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [13]:
 inputs, targets = next(iter(train_ds))

 print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
 print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
 print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 20)
inputs["decoder_inputs"].shape: (64, 20)
targets.shape: (64, 20)


In [14]:
inputs["encoder_inputs"]

<tf.Tensor: shape=(64, 20), dtype=int64, numpy=
array([[109,   2,   5, ...,   0,   0,   0],
       [ 27,   4,   0, ...,   0,   0,   0],
       [  2,  87,  35, ...,   0,   0,   0],
       ...,
       [ 68,  28,   0, ...,   0,   0,   0],
       [133,  10,   0, ...,   0,   0,   0],
       [  8,  66,   0, ...,   0,   0,   0]])>

In [15]:
inputs["decoder_inputs"]

<tf.Tensor: shape=(64, 20), dtype=int64, numpy=
array([[  2,  89,  61, ...,   0,   0,   0],
       [  2, 363,   3, ...,   0,   0,   0],
       [  2,   4,   9, ...,   0,   0,   0],
       ...,
       [  2,  77,   3, ...,   0,   0,   0],
       [  2, 448,   3, ...,   0,   0,   0],
       [  2,   6,  27, ...,   0,   0,   0]])>

In [16]:
targets

<tf.Tensor: shape=(64, 20), dtype=int64, numpy=
array([[ 89,  61,  28, ...,   0,   0,   0],
       [363,   3,   0, ...,   0,   0,   0],
       [  4,   9,   5, ...,   0,   0,   0],
       ...,
       [ 77,   3,   0, ...,   0,   0,   0],
       [448,   3,   0, ...,   0,   0,   0],
       [  6,  27, 569, ...,   0,   0,   0]])>

In [17]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)

        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads

        self.attention = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = embed_dim
        )

        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation = "relu"),
                layers.Dense(embed_dim),
            ]
        )

        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

        self.supports_masking = True

    def call(self, inputs, mask = None):
        attention_output = self.attention(query = inputs, value = inputs, key = inputs)

        proj_input = self.layernorm_1(inputs + attention_output)

        proj_output = self.dense_proj(proj_input)

        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()

        config.update(
            {
                "embed_dim": self.embed_dim,
                "dense_dim": self.dense_dim,
                "num_heads": self.num_heads,
            }
        )

        return config

In [27]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)

        self.token_embeddings = layers.Embedding(
            input_dim = vocab_size, output_dim = embed_dim
        )

        self.position_embeddings = layers.Embedding(
            input_dim = sequence_length, output_dim = embed_dim
        )

        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start = 0, limit = length, delta = 1)

        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)

        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask = None):
        # return tf.math.not_equal(inputs, 0)
         return layers.Lambda(lambda x: tf.not_equal(tf.cast(x, tf.int64), 0))(inputs)

    def get_config(self):
        config = super().get_config()

        config.update(
            {
                "sequence_length": self.sequence_length,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )

        return config

In [28]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)

        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads

        self.attention_1 = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = embed_dim
        )

        self.dense_proj = keras.Sequential(
            [
                layers.Dense(latent_dim, activation = "relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()

        self.add = layers.Add()  # instead of `+` to preserve mask

        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask = None):
        attention_output_1 = self.attention_1(
            query = inputs, value = inputs, key = inputs, use_causal_mask = True
        )
        out_1 = self.layernorm_1(self.add([inputs, attention_output_1]))

        attention_output_2 = self.attention_2(
            query = out_1,
            value = encoder_outputs,
            key = encoder_outputs,
        )
        out_2 = self.layernorm_2(self.add([out_1, attention_output_2]))

        proj_output = self.dense_proj(out_2)

        return self.layernorm_3(self.add([out_2, proj_output]))

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "latent_dim": self.latent_dim,
                "num_heads": self.num_heads,
            }
        )
        return config

In [29]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

# Encoder
encoder_inputs = keras.Input(shape = (None,), dtype = "int64", name = "encoder_inputs")

x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)

encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)

encoder = keras.Model(encoder_inputs, encoder_outputs)

# Decoder
decoder_inputs = keras.Input(shape = (None,), dtype = "int64", name = "decoder_inputs")

encoded_seq_inputs = keras.Input(shape = (None, embed_dim), name = "decoder_state_inputs")

x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)

x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)

x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation = "softmax")(x)

decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])

# Transformer
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name = "transformer"
)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'functional_3' (of type Functional) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


In [30]:
epochs = 30  # This should be at least 30 for convergence

transformer.summary()

transformer.compile(
    "rmsprop", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]
)

transformer.fit(train_ds, epochs = epochs, validation_data = val_ds)

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ positional_embedding_3    │ (None, None, 256)      │      3,845,120 │ encoder_inputs[0][0]   │
│ (PositionalEmbedding)     │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda (Lambda)           │ (None, None)           │              0 │ encoder_inputs[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder       │ (None, None, 256)      │      3,155,456 │ positional_embedding_… │
│ (TransformerEncoder)      │                        │                │ lambda[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_3 (Functional) │ (None, None, 15000)    │     12,959,640 │ decoder_inputs[0][0],  │
│                           │                        │                │ transformer_encoder[0… │
│                           │                        │                │ lambda[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 19,960,216 (76.14 MB)

 Trainable params: 19,960,216 (76.14 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/ke

11/11 ━━━━━━━━━━━━━━━━━━━━ 67s 5s/step - accuracy: 0.0327 - loss: 8.7975 - val_accuracy: 0.0607 - val_loss: 7.1932
Epoch 2/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 77s 5s/step - accuracy: 0.0554 - loss: 6.6723 - val_accuracy: 0.0517 - val_loss: 6.0434
Epoch 3/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step - accuracy: 0.0510 - loss: 5.5097 - val_accuracy: 0.0477 - val_loss: 5.4029
Epoch 4/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step - accuracy: 0.0514 - loss: 4.7894 - val_accuracy: 0.0670 - val_loss: 5.0768
Epoch 5/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 57s 5s/step - accuracy: 0.0556 - loss: 4.3827 - val_accuracy: 0.0583 - val_loss: 4.7956
Epoch 6/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 64s 6s/step - accuracy: 0.0589 - loss: 3.8696 - val_accuracy: 0.0590 - val_loss: 4.4964
Epoch 7/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 75s 5s/step - accuracy: 0.0646 - loss: 3.4435 - val_accuracy: 0.0477 - val_loss: 4.4164
Epoch 8/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 81s 5s/step - accuracy: 0.0648 - loss: 3.1639 - val_accuracy: 0.0663 - val_loss: 4.0796
Epo

In [31]:
fra_vocab = fra_vectorization.get_vocabulary()
fra_index_lookup = dict(zip(range(len(fra_vocab)), fra_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"


    for i in range(max_decoded_sentence_length):
        # Tokenize the sentence generated so far
        tokenized_target_sentence = fra_vectorization([decoded_sentence])[:, :-1]

        # Predict the next word in the sequence using input sentence and
        # tokenized representation of sentence so far
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        # Look up the index of the current predicted word
        sampled_token_index = np.argmax(predictions[0, i, :])

        # Find the corresponding token from the vocabulary
        sampled_token = fra_index_lookup[sampled_token_index]

        # Add the predicted word to the decoded text so far
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break

    return decoded_sentence

In [32]:
test_eng_texts = [pair[0] for pair in test_pairs]

for _ in range(30):
    ip = random.choice(test_eng_texts)
    print("input :",ip)
    print("translation :", decode_sequence(ip))

input : I'm tidy.
translation : [start] jeus [end]
input : Stay down.
translation : [start] aller baissé  salut salut salut salut Ça vous salut vous vous vous en salut en vous salut salut vous
input : Go inside.
translation : [start] jadore suis  vous salut vous suis Ça vous salut vous suis vous vous salut en suis salut salut vous
input : Really?
translation : [start] jadore oh  vous salut vous salut Ça vous salut vous vous vous vous salut en vous salut salut vous
input : I'll call.
translation : [start] y appelle  vous salut vous salut Ça vous salut vous vous vous vous salut en vous salut salut vous
input : Come on.
translation : [start] ici quittez ici ici ici ici ici ici ici ici ici ici ici ici ici ici ici ici ici ici
input : Go home.
translation : [start] jabandonne suis  vous salut vous suis Ça vous salut vous suis vous vous salut en suis salut salut vous
input : Let's go!
translation : [start] je suis  suis salut vous suis suis vous suis vous suis vous vous salut suis suis salut 

In [33]:
for test_pair in test_pairs[:30]:
    input_sentence = test_pair[0]
    reference_sentence = test_pair[1]

    translated_sentence = decode_sequence(input_sentence)
    translated_sentence = (
        translated_sentence.replace("[PAD]", "")
        .replace("[start]", "")
        .replace("[end]", "")
        .strip()
    )

    reference_sentence = (
        reference_sentence.replace("[PAD]", "")
        .replace("[start]", "")
        .replace("[end]", "")
        .strip()
    )

    print("English: ", input_sentence)
    print("Translation: ", reference_sentence)
    print("Prediction: ", translated_sentence)

English:  Tom walks.
Translation:  Tom marche.
Prediction:  allez marche
English:  I am fat.
Translation:  Je suis gras.
Prediction:  à plus
English:  Come over.
Translation:  Viens chez nous !
Prediction:  jen a  vous salut vous salut Ça vous salut vous vous vous vous salut en vous salut salut vous
English:  I'll cook.
Translation:  Je cuisinerai.
Prediction:  jadore moi
English:  Use this.
Translation:  Utilisez ceci.
Prediction:  de ceci de de de de de de ceci de de ceci de de ceci de ceci de de de
English:  Take mine.
Translation:  Prends le mien.
Prediction:  restez la  vous salut vous salut Ça vous vous vous vous vous vous salut vous vous salut salut vous
English:  I'm clean.
Translation:  J'ai décroché.
Prediction:  jeus suis  vous salut vous salut Ça vous salut vous suis vous vous salut en vous salut salut vous
English:  Keep this.
Translation:  Gardez ceci.
Prediction:  de ceci de ceci de ceci de ceci de de ceci de de de ceci de ceci de ceci de
English:  Wow!
Translation:  Ça 